In [1]:
# 🔧 Importa as bibliotecas necessárias
import pandas as pd                                # Para trabalhar com tabelas
import requests                                    # Para fazer chamadas à API
import time                                        # Para controlar o tempo entre chamadas
from urllib.parse import quote_plus               # Para codificar nomes em URLs
from requests.exceptions import RequestException  # Para capturar erros da API
from unidecode import unidecode                   # Para remover acentos das cidades

# 📁 1. Leitura do arquivo CSV com os dados dos países
df = pd.read_csv("df_fato_pais.csv")  # Certifique-se de que o arquivo está no mesmo diretório do script

# 🧹 2. Filtra os dados válidos: capital e código do país (sigla de 2 letras como BR, US, etc.)
cidades_paises = [
    (row['capital'], row['nome_resumido'])
    for _, row in df.iterrows()
    if (
        pd.notna(row['capital']) and                      # Capital não pode ser ausente
        pd.notna(row['nome_resumido']) and                # Código do país também não
        row['capital'].strip().lower() != 'não informado' and
        row['capital'].strip() != ''                      # Capital não pode estar vazia
    )
]

# 🧼 3. Remove duplicatas, caso algum país tenha aparecido mais de uma vez
cidades_paises = list(set(cidades_paises))

# 🌡️ 4. Lista onde vamos guardar os dados climáticos coletados
dados_clima = []

# ❌ 5. Lista de capitais que falharam ao buscar o clima (útil para revisar depois)
capitais_falharam = []

# 🔑 6. Sua chave da API da OpenWeatherMap (você precisa se cadastrar no site deles para conseguir)
api_key = "e0b696d684ea5b8a15f7f141ca979bc3"  # <<<<<<<<<<<<<<< Substitua isso pela sua chave real

# 🔁 7. Loop para buscar o clima de cada cidade
for cidade_original, codigo_pais in cidades_paises:

    # 🔤 Remove acentos da cidade (ex: "Brasília" → "Brasilia")
    cidade_sem_acento = unidecode(cidade_original)

    # 🌐 Codifica para URL (ex: "San Jose" → "San+Jose")
    cidade_codificada = quote_plus(cidade_sem_acento)

    try:
        # 🔗 Monta os parâmetros para chamada da API
        url = "https://api.openweathermap.org/data/2.5/weather"
        params = {
            "q": f"{cidade_codificada},{codigo_pais}",  # Ex: "Brasilia,BR"
            "appid": api_key,
            "units": "metric",                          # Para temperatura em graus Celsius
            "lang": "pt_br"                             # Retorno em português
        }

        # 🌐 Chama a API com timeout de 10 segundos
        resposta = requests.get(url, params=params, timeout=10)
        resposta.raise_for_status()  # Se a resposta for erro (ex: 404), levanta exceção

        # 🔄 Converte a resposta JSON da API para um dicionário Python
        clima_info = resposta.json()

        # ✅ Salva os dados relevantes da resposta na lista
        dados_clima.append({
            "cidade": cidade_original,
            "codigo_pais": clima_info['sys']['country'],
            "temperatura": clima_info['main']['temp'],
            "sensacao_termica": clima_info['main']['feels_like'],
            "temperatura_min": clima_info['main']['temp_min'],
            "temperatura_max": clima_info['main']['temp_max'],
            "umidade": clima_info['main']['humidity'],
            "descricao_clima": clima_info['weather'][0]['description'],
            "horario_medicao": clima_info['dt']  # Timestamp do horário da medição
        })

    except RequestException as e:
        # ⚠️ Caso ocorra erro da API (ex: cidade não encontrada)
        print(f"[ERRO API] {cidade_original}, {codigo_pais}: {e}")
        capitais_falharam.append((cidade_original, codigo_pais))

    except Exception as e:
        # ⚠️ Qualquer outro erro inesperado
        print(f"[ERRO INESPERADO] {cidade_original}, {codigo_pais}: {e}")
        capitais_falharam.append((cidade_original, codigo_pais))

    # ⏱️ Espera 1 segundo para não ultrapassar limite da API
    time.sleep(1)

# 📊 8. Converte a lista de dados em uma tabela
df_clima = pd.DataFrame(dados_clima)

# 🖨️ 9. Mostra os dados climáticos coletados
print("\n📋 Dados coletados:")
print(df_clima)

# ❗ 10. Exibe as capitais que não foram encontradas
print("\n❌ Capitais com erro:")
for cidade, pais in capitais_falharam:
    print(f" - {cidade} ({pais})")

# 💾 11. (Opcional) Salva os dados em arquivos CSV
df_clima.to_csv("df_clima_capitais.csv", index=False)
pd.DataFrame(capitais_falharam, columns=["cidade", "codigo_pais"]).to_csv("df_falhas_clima.csv", index=False)


[ERRO API] King Edward Point, GS: 404 Client Error: Not Found for url: https://api.openweathermap.org/data/2.5/weather?q=King%2BEdward%2BPoint%2CGS&appid=e0b696d684ea5b8a15f7f141ca979bc3&units=metric&lang=pt_br
[ERRO API] Ngerulmud, PW: 404 Client Error: Not Found for url: https://api.openweathermap.org/data/2.5/weather?q=Ngerulmud%2CPW&appid=e0b696d684ea5b8a15f7f141ca979bc3&units=metric&lang=pt_br
[ERRO API] Nuku'alofa, TO: 404 Client Error: Not Found for url: https://api.openweathermap.org/data/2.5/weather?q=Nuku%2527alofa%2CTO&appid=e0b696d684ea5b8a15f7f141ca979bc3&units=metric&lang=pt_br
[ERRO API] San José, CR: 404 Client Error: Not Found for url: https://api.openweathermap.org/data/2.5/weather?q=San%2BJose%2CCR&appid=e0b696d684ea5b8a15f7f141ca979bc3&units=metric&lang=pt_br
[ERRO API] Washington DC, UM: 404 Client Error: Not Found for url: https://api.openweathermap.org/data/2.5/weather?q=Washington%2BDC%2CUM&appid=e0b696d684ea5b8a15f7f141ca979bc3&units=metric&lang=pt_br
[ERRO API

In [2]:

display(df_clima)


,cidade,codigo_pais,temperatura,sensacao_termica,temperatura_min,temperatura_max,umidade,descricao_clima,horario_medicao
0,Kinshasa,CD,32.21,34.27,32.18,32.21,48,algumas nuvens,1758820859
1,Bangui,CF,22.71,23.52,22.71,22.71,95,nublado,1758820682
2,Lilongwe,MW,26.58,26.58,26.58,26.58,25,nuvens dispersas,1758820862
3,Antananarivo,MG,20.98,20.60,20.98,20.98,56,algumas nuvens,1758820863
4,Tegucigalpa,HN,26.92,27.78,26.92,27.42,57,nuvens dispersas,1758820800
...,...,...,...,...,...,...,...,...,...
191,Seoul,KR,17.76,18.20,16.78,17.76,100,névoa,1758821114
192,Skopje,MK,22.82,22.44,22.82,22.82,49,nuvens dispersas,1758821210
193,Belmopan,BZ,33.06,40.06,33.06,33.06,63,nublado,1758821211
194,Algiers,DZ,22.90,23.05,22.90,22.90,69,chuva leve,1758820939


In [3]:
from pandasql import sqldf

# Define uma função para executar consultas SQL
pysqldf = lambda q: sqldf(q, globals())

In [4]:
consultaSQL = """

SELECT * FROM df_clima 
--where codigo_pais = "AR"

"""

resultado = pysqldf(consultaSQL)
display(resultado)

,cidade,codigo_pais,temperatura,sensacao_termica,temperatura_min,temperatura_max,umidade,descricao_clima,horario_medicao
0,Kinshasa,CD,32.21,34.27,32.18,32.21,48,algumas nuvens,1758820859
1,Bangui,CF,22.71,23.52,22.71,22.71,95,nublado,1758820682
2,Lilongwe,MW,26.58,26.58,26.58,26.58,25,nuvens dispersas,1758820862
3,Antananarivo,MG,20.98,20.60,20.98,20.98,56,algumas nuvens,1758820863
4,Tegucigalpa,HN,26.92,27.78,26.92,27.42,57,nuvens dispersas,1758820800
...,...,...,...,...,...,...,...,...,...
191,Seoul,KR,17.76,18.20,16.78,17.76,100,névoa,1758821114
192,Skopje,MK,22.82,22.44,22.82,22.82,49,nuvens dispersas,1758821210
193,Belmopan,BZ,33.06,40.06,33.06,33.06,63,nublado,1758821211
194,Algiers,DZ,22.90,23.05,22.90,22.90,69,chuva leve,1758820939


In [5]:
resultado['horario_medicao'] = pd.to_datetime(resultado['horario_medicao'], unit='s').dt.strftime('%d/%m/%Y %H:%M:%S')

In [6]:
display(resultado)

,cidade,codigo_pais,temperatura,sensacao_termica,temperatura_min,temperatura_max,umidade,descricao_clima,horario_medicao
0,Kinshasa,CD,32.21,34.27,32.18,32.21,48,algumas nuvens,25/09/2025 17:20:59
1,Bangui,CF,22.71,23.52,22.71,22.71,95,nublado,25/09/2025 17:18:02
2,Lilongwe,MW,26.58,26.58,26.58,26.58,25,nuvens dispersas,25/09/2025 17:21:02
3,Antananarivo,MG,20.98,20.60,20.98,20.98,56,algumas nuvens,25/09/2025 17:21:03
4,Tegucigalpa,HN,26.92,27.78,26.92,27.42,57,nuvens dispersas,25/09/2025 17:20:00
...,...,...,...,...,...,...,...,...,...
191,Seoul,KR,17.76,18.20,16.78,17.76,100,névoa,25/09/2025 17:25:14
192,Skopje,MK,22.82,22.44,22.82,22.82,49,nuvens dispersas,25/09/2025 17:26:50
193,Belmopan,BZ,33.06,40.06,33.06,33.06,63,nublado,25/09/2025 17:26:51
194,Algiers,DZ,22.90,23.05,22.90,22.90,69,chuva leve,25/09/2025 17:22:19


In [7]:
df_export = pd.DataFrame(resultado)  # Dados dim pais
df_export.to_csv("df_dim_tempo_capital.csv", index=False)

In [8]:
# ==========================================================
# Importando bibliotecas necessárias
# ==========================================================
import os
from sqlalchemy import create_engine
from dotenv import load_dotenv, find_dotenv

# Carrega as variáveis de ambiente a partir do arquivo .env
load_dotenv(find_dotenv())

# ==========================================================
# Definindo as credenciais do banco (Render)
# ==========================================================
host = os.getenv("DB_HOST_PROD")
port = int(os.getenv("DB_PORT_PROD"))
database = os.getenv("DB_NAME_PROD")
user = os.getenv("DB_USER_PROD")
password = os.getenv("DB_PASS_PROD")

# ==========================================================
# Criando a conexão com o PostgreSQL usando SQLAlchemy
# ==========================================================
engine = create_engine(
    f"postgresql://{user}:{password}@{host}:{port}/{database}"
)


# ==========================================================
# Salvando no banco (tabela: clima)
# if_exists:
#   - 'replace' -> apaga e recria a tabela
#   - 'append' -> adiciona os dados
# ==========================================================
df_export.to_sql("tbl_clima_capital_pais", engine, if_exists="replace", index=False)

print("✅ Dados salvos no banco Render com sucesso!")

✅ Dados salvos no banco Render com sucesso!


In [9]:
import psycopg2

# Parâmetros de conexão
dbname   = 'projeto_paises'
user     = 'postgres'
password = '252500'
host     = 'localhost'
port     = '5432' 

# Criar uma conexão
conn = psycopg2.connect(dbname=dbname,
                        user=user,
                        password=password,
                        host=host,
                        port=port)


cur = conn.cursor()# Criar um cursor  deixa manipular os dados
#cur.execute("DELETE FROM TBL_CLIMA_CAPITAL_PAIS")

for indice, colunas_df in resultado.iterrows():
        cur.execute('''                                
                insert into TBL_CLIMA_CAPITAL_PAIS (   
                 cidade,
                 codigo_pais,
                 temperatura,
                 sensacao_termica,
                 temperatura_min,
                 temperatura_max,
                 umidade,
                 descricao_clima,
                 horario_medicao)
                 VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)
                        ''',
        (
        colunas_df["cidade"],
        colunas_df["codigo_pais"],
        colunas_df["temperatura"],
        colunas_df["sensacao_termica"],
        colunas_df["temperatura_min"],
        colunas_df["temperatura_max"],
        colunas_df["umidade"],
        colunas_df["descricao_clima"],
        colunas_df["horario_medicao"]
        )
        )

#comandos sql transct-sql
conn.commit() # validar alterações que agente fez  e subir para o banco de dados 

# Fechar o cursor e a conexão
cur.close()
conn.close()